## Chatbot using Persistence from Langgraph
- Install required python packages locally
- `pip install -qU langchain-openai`
- `pip install -U langgraph`

### First install basic items required to define LLM

In [16]:
import os
from dotenv import load_dotenv, find_dotenv

# Load local env file
_ = load_dotenv(find_dotenv())

from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-3.5-turbo", api_key=os.environ['OPENAI_API_KEY'])


## Message Persistence using Langgraph
- An alternative approach is written under the next major heading : Prompt Template

In [27]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a function to call the model
async def invoke_model(state: MessagesState):
    response = await model.ainvoke(state["messages"])
    return {"messages": response}

# Define a new graph
chatflow = StateGraph(state_schema=MessagesState)

# Add first edge to the graph
chatflow.add_edge(START, "model")

# Add a node to the edge of graph
chatflow.add_node("model", invoke_model)

# Add memory
app = chatflow.compile(checkpointer=MemorySaver())
 

#### Enter the first set of input

In [ ]:

# Use config to manage session
config = {"configurable": {"thread_id": "utkal"}}

from langchain_core.messages import HumanMessage
user_query = "Hi My name is Utkal !"
input_msg = [HumanMessage(user_query)]
output_msg = await app.ainvoke({"messages": input_msg},config)
output_msg["messages"][-1].pretty_print()  

================================== Ai Message ==================================

Hello Utkal! How can I assist you today?


#### Second set of input using the same config as first input set

In [29]:
user_query = "What is my name ?"
input_msg = [HumanMessage(user_query)]
output_msg = await app.ainvoke({"messages": input_msg},config)
output_msg["messages"][-1].pretty_print() 

================================== Ai Message ==================================

Your name is Utkal!


#### Third set of input using a different config from first input set

In [30]:

# Use config to manage session
config = {"configurable": {"thread_id": "utkal1"}}

user_query = "What is my name ?"
input_msg = [HumanMessage(user_query)]
output_msg = await app.ainvoke({"messages": input_msg},config)
output_msg["messages"][-1].pretty_print() 

================================== Ai Message ==================================

I'm sorry, I do not have access to your personal information and therefore do not know your name.


### Implement above prompts using prompt template

In [35]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
system_template = "You are a helpful software engineer with deep knowledge of cloud computing. Answer the question asked by user in a concise way with one example. Do not provide any imaginary answer. The user input is given here :"
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system",system_template),
        MessagesPlaceholder(variable_name="messages"),
    ]

)

# Define a function to call the model
async def invoke_chain_model(state: MessagesState):
    chain = prompt_template | model 
    response = await chain.ainvoke(state["messages"])
    return {"messages": response}

# Define a new graph
chatflowPT = StateGraph(state_schema=MessagesState)

# Add first edge to the graph
chatflowPT.add_edge(START, "model")

# Add a node to the edge of graph
chatflowPT.add_node("model", invoke_chain_model)

# Add memory
appPT = chatflowPT.compile(checkpointer=MemorySaver())

#### Invoke the prompt template based chat call

In [ ]:
configPT = {"configurable":{"thread_id":"utkal2"}}
user_input = "Hi My name is Utkal Nayak !"
input_msg_pt = [HumanMessage(user_input)]
output = await appPT.ainvoke({"messages": input_msg_pt}, configPT)
output["messages"][-1].pretty_print()

#### Second input to check memory

In [ ]:
user_input = "What is my name ?"
input_msg_pt = [HumanMessage(user_input)]
output = await appPT.ainvoke({"messages": input_msg_pt}, configPT)
output["messages"][-1].pretty_print()

## Manage Chat History

In [ ]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_token = 65,
    strategy = "last",
    token_counter = "model",
    include_system = "true",
    allow_partial = "False",
    start_on = "human",
)


